In [0]:
data = [('Alice', 'Badminton, Tennis'), ('Bob', 'Tennis, Cricket'), ('Julie', 'Cricket, Carroms')]
columns=["Name", "Hobbies"]

In [0]:
df = spark.createDataFrame(data, columns)

display(df)

In [0]:
from pyspark.sql.functions import *

df1 = df.withColumn("Hobby", explode(split(col("Hobbies"), ","))).select(col("Name"), col("Hobby"))


display(df1)

In [0]:
data = [('Goa', '', 'AP'), ('', 'AP', None), (None, '', 'bglr')]
columns=["city1","city2","city3"]

cityDf = spark.createDataFrame(data, columns)

In [0]:
display(cityDf)

In [0]:
cityDf1 = cityDf.withColumn("city",coalesce(when(col("city1") == "", None).otherwise(col("city1")),when(col("city2") == "", None).otherwise(col("city2")),when(col("city3") == "", None).otherwise(col("city3"))))

display(cityDf1)

In [0]:
data1= [(1, "Steve"), (2, "David"), (3, "John"), (4, "Shree"), (5, "Helen")]
data2= [(1, "SQL",90), (1, "PySpark", 100), (2, "SQL",70), (2, "PySpark", 60), (3, "SQL", 30), (3, "PySpark", 20), (4, "SQL", 50), (4,
"PySpark", 50), (5, "SQL",45), (5, "PySpark", 45)]
schema1=["Id", "Name"]
schema2=["Id", "Subject", "Mark"]

In [0]:
dfsoft1 = spark.createDataFrame(data1, schema1)
dfsoft2 = spark.createDataFrame(data2, schema2)

In [0]:
display(dfsoft1)
display(dfsoft2)

In [0]:
dfMarks = dfsoft1.join(dfsoft2, "Id").groupBy(col("Id"), col("Name")).agg((sum(col("Mark"))/count(col("Mark"))).alias("percentage")).select(col("Id"), col("Name"), col("percentage"))

In [0]:
display(dfMarks)

In [0]:
dfMarks = dfMarks.withColumn("Grade", when(col("percentage") >= 90, "Distinction")
                   .when(col("percentage") >= 80, "First Class")
                   .when(col("percentage") >= 70, "Second Class")
                   .when(col("percentage") >= 60, "Third Class").otherwise("False"))
                   
display(dfMarks)

In [0]:
data1=[(1, "A", 1000, "IT"), (2, "B", 1500,"IT"), (3, "C", 2500,"IT"), (4, "D",3000, "HR"), (5, "E", 2000, "HR"), (6, "F", 1000, "HR")
, (7, "G", 4000, "Sales"), (8, "H", 4000, "Sales"), (9, "I", 1000, "Sales"), (10, "J", 2000, "Sales")]
schema1=["EmpId", "EmpName", "Salary", "DeptName"]

dfDept = spark.createDataFrame(data1, schema1)
display(dfDept)

In [0]:
from pyspark.sql.window import Window


window = Window.partitionBy("DeptName").orderBy(col("Salary").desc())
dfDept = dfDept.withColumn("Rank", dense_rank().over(window))
display(dfDept)


In [0]:
display(dfDept.where(col("Rank") == 1))


In [0]:
data1=[(100,"Raj", None, 1, "01-04-23", 50000),
(200, "Joanne", 100, 1, "01-04-23",4000), (200, "Joanne", 100, 1, "13-04-23",4500), (200, "Joanne", 100, 1, "14-04-23",4020)]
schema1=["EmpId", "EmpName", "Mgrid", "deptid","salarydt", "salary"]
dfStudents = spark.createDataFrame(data1, schema1)


data2=[(1, "IT"),
(2,"HR")]
schema2=["deptid", "deptname"]
dfDept = spark.createDataFrame(data2, schema2)
display(dfStudents)
display(dfDept)

In [0]:
from pyspark.sql.functions import *

dfStudents = dfStudents.withColumn("newsalarydt", to_date(col("salarydt"), "dd-MM-yy"))
# dfStudents = dfStudents.withColumn("salarydt1", to_date(col("salarydt"), "dd-MMM-yy"))
# dfStudents = dfStudents.withColumn("salarydt2", to_date(col("salarydt"), "dd-MMMM-yy"))
# dfStudents = dfStudents.withColumn("salarydt4", date_format(col("salarydt"), "yyyy-MM-dd"))
# dfStudents = dfStudents.withColumn("salarydt5", to_timestamp(col("salarydt"), "yyyy-MM-dd"))
dfStudents = dfStudents.drop(col("salarydt1"), col("salarydt2"), col("salarydt4"), col("salarydt5"))
display(dfStudents)

In [0]:
dfStudentsDept = dfStudents.join(dfDept, "deptid")

In [0]:
display(dfStudentsDept)

In [0]:
dfSelfJoin = dfStudentsDept.alias("a").join(dfStudentsDept.alias("b"), col("a.Mgrid") == col("b.EmpId"), "left").select(col("a.EmpName"),col("a.deptname"),col("a.salary"),col("a.newsalarydt"), col("b.EmpName").alias("ManagerName"))
display(dfSelfJoin)

In [0]:
dfnewJoin = dfSelfJoin.groupBy(col("deptname"), col("ManagerName"), col("EmpName")
                   , year(col("newsalarydt")).alias("Year")
                   , date_format(col("newsalarydt"), "MMM").alias("Month"
                   )).sum("salary").alias("TotalSalary")


display(dfnewJoin)

In [0]:
filePath = "/FileStore/sales_200_rows.csv"

In [0]:
dfsales = spark.read.csv(filePath, header=True, inferSchema=True)
display(dfsales)

In [0]:
dfsalesp = dfsales.repartition(11)

In [0]:
dfsalesp = dfsalesp.select(spark_partition_id().alias("partitionID")).groupBy("partitionID").count()
display(dfsalesp)

In [0]:
filepath1 = "/FileStore/nested_*.json"

In [0]:
import datetime

from pyspark.sql.types import StructType, StructField, StringType, LongType, BooleanType

schema = StructType([
    StructField("address", StructType([
        StructField("city", StringType(), True),
        StructField("number", LongType(), True),
        StructField("street", StringType(), True),
        StructField("zip", StringType(), True)
    ]), True),
    StructField("email", StringType(), True),
    StructField("error", StringType(), True),
    StructField("id", LongType(), True),
    StructField("name", StringType(), True),
    StructField("profile", StructType([
        StructField("active", BooleanType(), True),
        StructField("age", LongType(), True),
        StructField("join_date", StringType(), True),
        StructField("raw", StringType(), True)
    ]), True)
])


dfnestedJson = spark.read.schema(schema).option("multiline", True).json(filepath1)

display(dfnestedJson)

In [0]:
data1=[(1, "Ram", "Male", 100), (2, "Radhe", "Female", 200), (3, "John", "Male", 250)]
data2=[(101, "John", "Male", 100), (102, "Joanne", "Female", 250), (103, "Smith", "Male", 250)]
data3=[(1001, "Maxwell", "IT", 200), (2, "MSD","HR", 350), (3, "Virat", "IT", 300)]

schema1=["Id", "Name", "IGender", "Salary"]
schema2=["Id", "Name", "Gender", "Salary"]
schema3=["Id", "Name", "DeptName", "Salary"]

df1 = spark.createDataFrame(data1, schema1)
df2 = spark.createDataFrame(data2, schema2)
df3 = spark.createDataFrame(data3, schema3)
display(df1)
display(df2)
display(df3)


In [0]:
if df1.schema ==df3.schema:
    print("equal")

else:
    print("not equal")


In [0]:
print(list[set(df1.schema) - set(df3.schema)])

In [0]:
print(list[set(df3.schema) - set(df1.schema)])


In [0]:
allcolu = df1.columns +df3.columns
print(allcolu)

In [0]:
dff = spark.read.format("json").option("multiline", True).option("pathGlobFilter", "*.json").option("recursiveFileLookup", True).option("inferSchema", True).load("/FileStore/")

display(dff)

In [0]:
dff.select(col("email"), col("name")).write.option("header", True).mode("overwrite").csv("/FileStore/",compression="gzip", encoding="cp1252")